<a href="https://colab.research.google.com/github/KIQVinicius/Data-Science/blob/master/AnomaliasJob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
import numpy as np

In [0]:
dataset = pd.read_excel("MIPS.xlsx")

In [0]:
grupo_jobname = dataset[dataset['Valor_MIPS']>0].groupby('JOBNAME')

In [27]:
for job, dados in grupo_jobname:
  print('{} -> {}'.format(job, dados['Valor_MIPS'].mean()))

AATTSO -> 0.015600000000000003
ANTAS000 -> 0.32
ANTMAIN -> 0.010074906367041149
AOFBUILD -> 17.0775
AOFBUILE -> 27.8
AOPD -> 0.053530655391120535
AOPSTOP -> 0.01
APPC -> 0.02699999999999978
ARCABKUP -> 20.99446808510638
ARCACDSB -> 2.7308888888888894
ARCAMIGR -> 4.107547169811316
ARCARCVR -> 0.02
ASCH -> 0.010219298245614012
ASFPCHIN -> 0.047020833333333366
ASFPMSTR -> 0.09064583333333373
AUXVFR00 -> 0.44
AXR -> 0.13167553191489387
AXR04 -> 0.010000000000000004
B11B01EX -> 0.04666666666666667
B11MAHR0 -> 0.02
B14A30MR -> 0.44
B22338CO -> 0.16
B22448CO -> 0.08
B5FR0105 -> 0.07
B5FR010L -> 0.07
B5FR0116 -> 0.06
B5FR0148 -> 0.08
B5FR0617 -> 0.1366666666666667
B5FR2033 -> 0.25
B5FR203G -> 0.42
B5FR2201 -> 0.42
B5FR2202 -> 0.93
B5FR2203 -> 0.49
B5FR2205 -> 0.11
B5FR2211 -> 0.17
B5FR2219 -> 0.3
B5FR2256 -> 0.11
B5FR2272 -> 0.245
B5FR2294 -> 0.10777777777777778
B5FR2295 -> 0.5
B5FR229A -> 0.39
BB2543 -> 0.76
BC0189 -> 17.71
BC5028 -> 0.024705882352941175
BC5029 -> 0.5447058823529412
BD26A08 -

In [29]:
grupo_jobname.describe().round(2)

Valor_MIPS                                                
              count   mean   std    min    25%    50%    75%    max
JOBNAME                                                            
AATTSO         25.0   0.02  0.01   0.01   0.01   0.02   0.02   0.03
ANTAS000        1.0   0.32   NaN   0.32   0.32   0.32   0.32   0.32
ANTMAIN       267.0   0.01  0.00   0.01   0.01   0.01   0.01   0.02
AOFBUILD        4.0  17.08  6.70  11.20  11.31  17.02  22.79  23.07
AOFBUILE        1.0  27.80   NaN  27.80  27.80  27.80  27.80  27.80
...             ...    ...   ...    ...    ...    ...    ...    ...
F4KR2263        1.0   0.11   NaN   0.11   0.11   0.11   0.11   0.11
F4KR2266        1.0   0.06   NaN   0.06   0.06   0.06   0.06   0.06
F4KR2275        1.0   0.12   NaN   0.12   0.12   0.12   0.12   0.12
F4KR22A2        1.0   0.02   NaN   0.02   0.02   0.02   0.02   0.02
F4KR2703        2.0   0.57  0.00   0.57   0.57   0.57   0.57   0.57

[692 rows x 8 columns]

In [30]:
grupo_jobname['Valor_MIPS'].mean()

JOBNAME
AATTSO       0.015600
ANTAS000     0.320000
ANTMAIN      0.010075
AOFBUILD    17.077500
AOFBUILE    27.800000
              ...    
F4KR2263     0.110000
F4KR2266     0.060000
F4KR2275     0.120000
F4KR22A2     0.020000
F4KR2703     0.570000
Name: Valor_MIPS, Length: 692, dtype: float64

In [0]:
job_media = pd.DataFrame(grupo_jobname['Valor_MIPS'].mean())


In [0]:
job_media.reset_index(inplace=True)

In [0]:
tabela_final = dataset.iloc[0:0]
for index, row in job_media.iterrows():
  query = "JOBNAME == "+"'"+row['JOBNAME']+"'" + " and Valor_MIPS > "+str(row['Valor_MIPS'])
  tabela_aux = dataset.query(query)
  tabela_final = tabela_final.append(tabela_aux)

In [49]:
tabela_final.shape

(13521, 4)

In [45]:
tabela_final

,chave,Data_Hora,Valor_MIPS,JOBNAME
5,AATTSO01/06/2020 05:00,2020-06-01T05:00:00.0000000,0.02,AATTSO
29,AATTSO02/06/2020 05:00,2020-06-02T05:00:00.0000000,0.02,AATTSO
66,AATTSO14/05/2020 18:00,2020-05-14T18:00:00.0000000,0.03,AATTSO
90,AATTSO15/05/2020 18:00,2020-05-15T18:00:00.0000000,0.02,AATTSO
125,AATTSO17/05/2020 05:00,2020-05-17T05:00:00.0000000,0.02,AATTSO
...,...,...,...,...
149892,F4KR226626/05/2020 12:00,2020-05-26T12:00:00.0000000,0.06,F4KR2266
149916,F4KR227526/05/2020 12:00,2020-05-26T12:00:00.0000000,0.12,F4KR2275
149944,F4KR22A227/05/2020 16:00,2020-05-27T16:00:00.0000000,0.02,F4KR22A2
149968,F4KR270302/06/2020 16:00,2020-06-02T16:00:00.0000000,0.57,F4KR2703
